In [3]:
import os
import numpy as np 
import random 


In [4]:
shtech_root = '/media/ubuntu/MyHDataStor3/datasets/violence/UCF_and_Shanghai/ShanghaiTech/'

train_list = shtech_root  + 'splits/train.txt'
test_list  = shtech_root  + 'splits/test.txt'

train_lines = open(train_list).readlines() 
train_lines = [x.strip() for x in train_lines] 
train_lines.sort()
print(len(train_lines))
print(train_lines[:5])

test_lines = open(test_list).readlines() 
test_lines = [x.strip() for x in test_lines] 
test_lines.sort()
print(len(test_lines))
print(test_lines[:5])


238
['01_0014', '01_0016', '01_002', '01_0026', '01_0029']
199
['01_001', '01_0015', '01_0025', '01_0027', '01_0028']


In [8]:
test_file = shtech_root+ 'testing/features_32/01_0014.npy' 
feature = np.load(test_file)
print(feature.shape)


(32, 1024)


In [9]:
test_file = shtech_root+ 'all_rgbs/abnormal/01_0014.npy' 
feature2 = np.load(test_file)
print(feature2.shape)


(32, 1024)


In [10]:
diff = feature - feature2 
print(diff)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [12]:
import pickle 

with open(shtech_root+'frames.pkl', 'rb') as f:
    num_frames  = pickle.load(f)

print(num_frames)


{'13_001': 0, '03_0039': 481, '01_034': 0, '01_029': 0, '13_006': 0, '10_009': 0, '09_009': 0, '01_0063': 193, '07_002': 0, '07_001': 0, '11_0176': 337, '02_002': 0, '08_017': 0, '05_020': 0, '05_0020': 649, '03_0061': 241, '01_020': 0, '01_022': 0, '01_0138': 313, '06_0144': 241, '08_024': 0, '01_0053': 457, '06_005': 0, '05_021': 0, '05_061': 0, '02_014': 0, '02_008': 0, '12_0142': 601, '01_048': 0, '06_0147': 265, '07_0008': 457, '01_023': 0, '06_018': 0, '05_058': 0, '08_009': 0, '11_009': 0, '02_011': 0, '05_012': 0, '05_024': 0, '04_0046': 529, '01_083': 0, '04_008': 0, '06_013': 0, '01_050': 0, '06_002': 0, '05_050': 0, '06_0145': 217, '12_003': 0, '03_005': 0, '07_004': 0, '05_045': 0, '03_006': 0, '07_0049': 481, '01_032': 0, '08_048': 0, '08_035': 0, '01_045': 0, '05_037': 0, '08_038': 0, '03_004': 0, '12_002': 0, '05_057': 0, '01_0029': 313, '04_017': 0, '01_0027': 409, '01_075': 0, '08_037': 0, '08_005': 0, '07_0009': 313, '12_011': 0, '05_0024': 433, '09_001': 0, '05_006':

In [13]:
with open(shtech_root+'GT_anomaly.pkl', 'rb') as f:
    GT = pickle.load(f)

print(GT)

{'02_0161': {'gt': [69, 289]}, '03_0039': {'gt': [224, 228, 243, 355, 356, 419, 420, 452]}, '12_0152': {'gt': [50, 203, 204, 257, 258, 266, 267, 276, 277, 290, 291, 306, 307, 342, 343, 361]}, '03_0059': {'gt': [132, 279]}, '01_0029': {'gt': [134, 247]}, '05_0017': {'gt': [284, 417]}, '01_0056': {'gt': [264, 399]}, '08_0058': {'gt': [1, 53, 70, 172, 175, 252]}, '09_0057': {'gt': [144, 228]}, '08_0159': {'gt': [101, 232]}, '01_0135': {'gt': [168, 357]}, '07_0008': {'gt': [178, 186, 187, 312, 313, 366]}, '04_0003': {'gt': [407, 571, 577, 622, 633, 640, 641, 651]}, '02_0164': {'gt': [104, 203, 204, 295]}, '01_0014': {'gt': [155, 231]}, '08_0157': {'gt': [139, 270, 274, 284]}, '05_0019': {'gt': [245, 405]}, '08_0080': {'gt': [10, 75, 98, 207]}, '12_0151': {'gt': [1, 3, 25, 236]}, '07_0047': {'gt': [172, 239, 240, 377]}, '05_0021': {'gt': [150, 231]}, '01_0026': {'gt': [279, 399]}, '03_0036': {'gt': [192, 289, 301, 420]}, '03_0060': {'gt': [188, 272]}, '07_0007': {'gt': [130, 182, 183, 187, 

In [28]:
train_normal_videos = [] 
train_abnormal_videos = [] 

for l in train_lines:    
    print(l)    
    
    gt = GT[l]['gt']
    print(gt)
    
    if len(gt) == 0: #normal video 
        
        npy_file = shtech_root+'all_rgbs/Normal_Videos_event/{}.npy'.format(l)
        feature = np.load(npy_file) 
        num_snippets = feature.shape[0]
        
        l_new = 'Normal_Videos_event/{} {} -1'.format(l,num_snippets*16)
        
        train_normal_videos.append(l_new)
        
    else: 
        
        num_frame = num_frames[l] 
        
        l_new = 'abnormal/{}|{}'.format(l,num_frame)
        l_new = l_new + '|['
        
        num_pairs = len(gt)//2        
        for i in range(num_pairs):
            if i<num_pairs-1:
                l_new = l_new + str(gt[2*i]) + ', ' + str(gt[2*i+1])+', '
            else:
                l_new = l_new + str(gt[2*i]) + ', ' + str(gt[2*i+1])
        l_new = l_new + ']'    
        
        train_abnormal_videos.append(l_new)
        
print(len(train_normal_videos))
print(train_normal_videos[:5])
print(len(train_abnormal_videos))
print(train_abnormal_videos[:5])

train_normal_file = shtech_root+'train_normal.txt'
fp = open(train_normal_file,'w') 
for l in train_normal_videos:
    fp.write(l.split(' ')[0]+'\n')
fp.close()

train_abnormal_file = shtech_root+'train_anomaly.txt'
fp = open(train_abnormal_file,'w') 
for l in train_abnormal_videos:
    fp.write(l.split('|')[0]+'\n')
fp.close()


01_0014
[155, 231]
01_0016
[189, 280]
01_002
[]
01_0026
[279, 399]
01_0029
[134, 247]
01_003
[]
01_0030
[150, 350]
01_005
[]
01_0054
[146, 439]
01_0063
[81, 152]
01_007
[]
01_0073
[172, 263]
01_0076
[1, 29, 51, 252]
01_009
[]
01_010
[]
01_011
[]
01_0129
[48, 146]
01_013
[]
01_0130
[128, 222]
01_0131
[69, 166]
01_0132
[117, 204]
01_0133
[1, 194, 195, 217]
01_0134
[91, 193, 229, 312]
01_0138
[116, 236]
01_014
[]
01_0140
[120, 193]
01_015
[]
01_016
[]
01_0163
[102, 222]
01_017
[]
01_019
[]
01_020
[]
01_022
[]
01_023
[]
01_024
[]
01_025
[]
01_029
[]
01_031
[]
01_033
[]
01_035
[]
01_036
[]
01_037
[]
01_041
[]
01_042
[]
01_044
[]
01_046
[]
01_048
[]
01_049
[]
01_051
[]
01_054
[]
01_060
[]
01_062
[]
01_063
[]
01_066
[]
01_067
[]
01_068
[]
01_069
[]
01_070
[]
01_073
[]
01_083
[]
02_002
[]
02_004
[]
02_005
[]
02_006
[]
02_010
[]
02_011
[]
02_0128
[95, 211, 256, 392]
02_013
[]
02_014
[]
02_0164
[104, 203, 204, 295]
03_001
[]
03_003
[]
03_0035
[61, 106, 226, 287]
03_0036
[192, 289, 301, 420]
03_0

In [30]:
test_normal_videos = [] 
test_abnormal_videos = [] 

for l in test_lines:    
    print(l)    
    
    gt = GT[l]['gt']
    print(gt)
    
    if len(gt) == 0: #normal video 
        
        npy_file = shtech_root+'all_rgbs/Normal_Videos_event/{}.npy'.format(l)
        feature = np.load(npy_file) 
        num_snippets = feature.shape[0]
        
        l_new = 'Normal_Videos_event/{} {} -1'.format(l,num_snippets*16)
        
        test_normal_videos.append(l_new)
        
    else: 
        
        num_frame = num_frames[l] 
        
        l_new = 'abnormal/{}|{}'.format(l,num_frame)
        l_new = l_new + '|['
        
        num_pairs = len(gt)//2        
        for i in range(num_pairs):
            if i<num_pairs-1:
                l_new = l_new + str(gt[2*i]) + ', ' + str(gt[2*i+1])+', '
            else:
                l_new = l_new + str(gt[2*i]) + ', ' + str(gt[2*i+1])
        l_new = l_new + ']'    
        
        test_abnormal_videos.append(l_new)
        
print(len(test_normal_videos))
print(test_normal_videos[:5])
print(len(test_abnormal_videos))
print(test_abnormal_videos[:5])

test_normal_file = shtech_root+'test_normal.txt'
fp = open(test_normal_file,'w') 
for l in test_normal_videos:
    fp.write(l+'\n')
fp.close()

test_abnormal_file = shtech_root+'test_anomaly.txt'
fp = open(test_abnormal_file,'w') 
for l in test_abnormal_videos:
    fp.write(l+'\n')
fp.close()


01_001
[]
01_0015
[87, 375]
01_0025
[149, 204]
01_0027
[78, 150, 151, 153]
01_0028
[267, 330, 331, 332]
01_004
[]
01_0051
[99, 203, 261, 327]
01_0052
[120, 130, 154, 233]
01_0053
[81, 86, 87, 317]
01_0055
[69, 313]
01_0056
[264, 399]
01_006
[]
01_0064
[77, 289]
01_008
[]
01_012
[]
01_0135
[168, 357]
01_0136
[134, 272, 273, 324, 325, 447]
01_0139
[86, 193]
01_0141
[79, 225]
01_0162
[58, 162]
01_0177
[107, 190, 191, 193]
01_018
[]
01_021
[]
01_026
[]
01_027
[]
01_028
[]
01_030
[]
01_032
[]
01_034
[]
01_038
[]
01_039
[]
01_040
[]
01_043
[]
01_045
[]
01_047
[]
01_050
[]
01_052
[]
01_053
[]
01_055
[]
01_056
[]
01_057
[]
01_058
[]
01_059
[]
01_061
[]
01_064
[]
01_065
[]
01_071
[]
01_072
[]
01_074
[]
01_075
[]
01_076
[]
01_077
[]
01_078
[]
01_079
[]
01_080
[]
01_081
[]
01_082
[]
02_001
[]
02_003
[]
02_007
[]
02_008
[]
02_009
[]
02_012
[]
02_015
[]
02_0161
[69, 289]
03_002
[]
03_0031
[328, 388]
03_0032
[145, 265]
03_0033
[52, 162, 163, 168]
03_005
[]
03_0059
[132, 279]
04_0004
[427, 557]
04_00

In [36]:
RTFM_SH_feature_dir = '/media/ubuntu/MyHDataStor3/datasets/violence/RTMF/SH_Test_ten_crop_i3d/'

test_normal_file = shtech_root+'test_normal_v2.txt'
fp = open(test_normal_file,'w') 
for l in test_normal_videos:
    print(l)
    
    vid = l.split(' ')[0].split('/')[-1] 
    
    RTFM_feature = np.load(RTFM_SH_feature_dir+'{}_i3d.npy'.format(vid))
    
    num_frame = RTFM_feature.shape[0]*16 
    
    l_new = '{} {} -1'.format(l.split(' ')[0],num_frame) 
    
    print(l_new)
    
    
    fp.write(l_new+'\n')
fp.close()

Normal_Videos_event/01_001 512 -1
Normal_Videos_event/01_001 768 -1
Normal_Videos_event/01_004 512 -1
Normal_Videos_event/01_004 784 -1
Normal_Videos_event/01_006 512 -1
Normal_Videos_event/01_006 512 -1
Normal_Videos_event/01_008 512 -1
Normal_Videos_event/01_008 1024 -1
Normal_Videos_event/01_012 512 -1
Normal_Videos_event/01_012 752 -1
Normal_Videos_event/01_018 512 -1
Normal_Videos_event/01_018 848 -1
Normal_Videos_event/01_021 512 -1
Normal_Videos_event/01_021 1024 -1
Normal_Videos_event/01_026 512 -1
Normal_Videos_event/01_026 736 -1
Normal_Videos_event/01_027 512 -1
Normal_Videos_event/01_027 944 -1
Normal_Videos_event/01_028 512 -1
Normal_Videos_event/01_028 960 -1
Normal_Videos_event/01_030 512 -1
Normal_Videos_event/01_030 976 -1
Normal_Videos_event/01_032 512 -1
Normal_Videos_event/01_032 1200 -1
Normal_Videos_event/01_034 512 -1
Normal_Videos_event/01_034 896 -1
Normal_Videos_event/01_038 512 -1
Normal_Videos_event/01_038 496 -1
Normal_Videos_event/01_039 512 -1
Normal_Vide

Normal_Videos_event/08_048 800 -1
Normal_Videos_event/09_002 512 -1
Normal_Videos_event/09_002 832 -1
Normal_Videos_event/09_003 512 -1
Normal_Videos_event/09_003 528 -1
Normal_Videos_event/09_005 512 -1
Normal_Videos_event/09_005 496 -1
Normal_Videos_event/09_007 512 -1
Normal_Videos_event/09_007 1520 -1
Normal_Videos_event/10_004 512 -1
Normal_Videos_event/10_004 1760 -1
Normal_Videos_event/10_005 512 -1
Normal_Videos_event/10_005 1792 -1
Normal_Videos_event/10_008 512 -1
Normal_Videos_event/10_008 1184 -1
Normal_Videos_event/10_009 512 -1
Normal_Videos_event/10_009 608 -1
Normal_Videos_event/10_011 512 -1
Normal_Videos_event/10_011 752 -1
Normal_Videos_event/11_003 512 -1
Normal_Videos_event/11_003 656 -1
Normal_Videos_event/11_004 512 -1
Normal_Videos_event/11_004 256 -1
Normal_Videos_event/11_005 512 -1
Normal_Videos_event/11_005 1072 -1
Normal_Videos_event/11_006 512 -1
Normal_Videos_event/11_006 544 -1
Normal_Videos_event/12_001 512 -1
Normal_Videos_event/12_001 416 -1
Normal_Vi